<a href="https://colab.research.google.com/github/deClot/Hakaton_Superhero/blob/master/3_Income.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
salary_nominal_otrals.rename(columns={'Unnamed: 0': 'otrasl'}, inplace=True)
salary_nominal_otrals.otrasl.value_counts()

    производство прочей неметаллической минеральной продукции                                                                                 1
  из нее научные исследования и разработки                                                                                                    1
    обработка древесины и производство изделий из дерева и пробки, кроме мебели, производство изделий из соломки и материалов для плетения    1
обеспечение электрической энергией, газом и паром; кондиционирование воздуха                                                                  1
образование                                                                                                                                   1
производство электрического оборудования                                                                                                      1
    производство бумаги и бумажных изделий                                                                                              

# https://www.gks.ru/labor_market_employment_salaries?print=1
# https://www.gks.ru/folder/13397

In [152]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
from plotly.offline import init_notebook_mode, iplot

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt

import IPython

def display(*dfs):
    for df in dfs:
        IPython.display.display(df)
 
def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
  '''))
  init_notebook_mode(connected=False)


plt.rcParams['figure.figsize'] = [10,5]

In [162]:
def get_season(x):
  return 1 if x in [12,1,2] else 2 if x in [3,4,5] else 3 if x in [6,7,8] else 4

def preprocess_datetime(df, time_columns):
    for t in time_columns:
        df[t+'_year'] = df[t].dt.year
        df[t+'_month'] = df[t].dt.month
        df[t+'_season'] = df[t+'_month'].apply(get_season)
        df[t+'_is_month_start'] = df[t].dt.is_month_start
        df[t+'_is_month_end'] = df[t].dt.is_month_end
        df[t+'_day'] = df[t].dt.day
        # df[t+'_hour'] = df[t].dt.hour
        df[t+'_dayofweek'] = df[t].dt.dayofweek
        df[t+'_is_weekend'] = df[t+'_dayofweek'] >= 5
        df[t+'_quarter'] = df[t].dt.quarter

def get_regions(df):
  base = {'Кукморский': 'Кукмор',
          'Мамадышский': 'Мамадыш',
          'Нурлатский': 'Нурлат',
          'Азнакаевский': 'Азнакаево', 
          'Апастовский':'Апастово',
          'Бугульминский': 'Бигульма',
          'Елабужский':'Елабуга',
          'Бавлинский':'Бавлы',
          'Нижегородский':'Нижний Новгород',
          'Чистопольский':'Чистополь',
          'Вятские поляны': 'Вятские Поляны',
          'Новгород Великий':'Новгород'}
  to_cut = {k:k[:-2]  for k in reg if 'ский' in k}

  temp = df.LivingRegionName.copy().astype('object')
  temp[temp.isin(base.keys())] = temp[temp.isin(base.keys())].map(base)
  temp[temp.isin(to_cut.keys())]  = temp[temp.isin(to_cut.keys())] .map(to_cut)
  temp = temp.map(city_to_regions)
  # temp[df.LivingRegionName=='Москва'] = 'Москва'
  assert temp.isna().sum() == 0

  df.LivingRegionName = temp.astype('category')

In [163]:
df = pd.read_excel('3_train.xlsx')


df.rename(columns={'ConfirmedMonthlyIncome (Target)': 'target'}, inplace=True)
df.drop('NaturalPersonID', axis=1, inplace=True)
df.kolichestvo_detej_mladshe_18 = df.kolichestvo_detej_mladshe_18.astype('object')

preprocess_datetime(df, ['RequestDate'])
df = df.drop('RequestDate', axis=1)
features_time = [f for f in df.columns if 'RequestDate_' in f]

features_cat =  df.select_dtypes('object').columns.tolist()
# df[features_cat] = df[features_cat].astype('category')
features_num = [f for f in df.columns if f not  in features_cat and f not in features_time]

assert len(features_num) + len(features_cat) + len(features_time) == df.shape[1]

df.drop(['SignIP.1', 'Employment.1','TypeOfWork.1'], axis=1, inplace=True)
for f in ['SignIP.1', 'Employment.1','TypeOfWork.1'] :
  features_cat.remove(f)

In [164]:
regions = pd.read_json('russia')
city_to_regions = regions.set_index('city').to_dict()['region']

reg = [f for f in df.LivingRegionName.unique() if 'ский' in f]
city = [f for f in df.LivingRegionName.unique() if f not in reg]
df['in_region'] = np.where(df.LivingRegionName.isin(reg), 1,0)
get_regions(df)

In [165]:
df['age_cat'] = pd.cut(df.age, bins=(0, 23, 30, 40, 50, 65, 100), labels=[0,1,2,3,4,5])
features_cat.append('age_cat')
features_num.remove('age')

df = df[(df.otrasl_rabotodatelya.notna())&(df.kolichestvo_rabotnikov_v_organizacii.notna())]
df.SignIP = df.SignIP.astype('object').fillna(0)#.astype('category')

df.TypeOfWork[df.SignIP=='ИП'] = 'Индивидуальный предприниматель'

df.Employment[df.SignIP=='ИП'] = 'Индивидуальный предприниматель'
df.Employment.fillna('нет данных', inplace=True)

# сотрудники банка работают на полную ставку - заполнение пропска по типу работы
df.loc[(df.IsBankWorker=='да')&(df.harakteristika_tekutschego_trudoustrojstva.isna()),
       'harakteristika_tekutschego_trudoustrojstva'] = 'Постоянная, полная занятость'
df.harakteristika_tekutschego_trudoustrojstva.isna().sum()

df.IncomeDocumentKind = df.IncomeDocumentKind.astype('object').fillna('Нет запрашивали')#.astype('category')
df.Residence = df.Residence.astype('object').fillna('na')#.astype('category')
df.TypeOfWork = df.TypeOfWork.astype('object').fillna('na')#.astype('category')
df['OrgNtoAge'] = (df.OrgStanding_N)/(df.age-20)

In [166]:
features_num.remove('target')

In [167]:
df.LivingRegionName = df.LivingRegionName.astype('object').str.replace('Башкирия', '')\
                                    .str.replace('Горьковская', 'обл.')\
                                    .str.replace('(', '').str.replace(')', '')\
                                    .str.replace('Чувашия', 'Чувашская Республика')\
                                    .str.replace('Удмуртия', 'Удмуртская Республика')\
                                    .str.replace('Татарстан', 'Республика Татарстан')\
                                    .str.replace('Марий Эл', 'Республика Марий Эл') \
                                    .str.replace('Башкортостан', 'Республика Башкортостан')\
                                    .str.replace('Санкт-Петербург и область', 'г.Санкт-Петербург')\
                                    .str.replace('Москва и Московская обл.', 'г.Москва')

# Income

In [208]:
salary_nominal_region = pd.read_excel('t2.xlsx')
salary_nominal_region_year = pd.read_excel('t4.xlsx')
salary_real_region_year_ration = pd.read_excel('t5.xlsx')
# salary_nominal_otrals = pd.read_excel('t6.xls', header=5)
# salaty_nominal_otrasl_year = pd.read_excel('t3.xlsx')
incom_region = pd.read_excel('urov_10sub-nm.xlsx', header=4)
# mrot = pd.read_excel('data.xls')

In [209]:
incom_region.rename(columns={'Unnamed: 0': 'region'}, inplace=True)
incom_region.columns = pd.MultiIndex.from_product([np.arange(2013,2021), (1,2,3,4,'total')]).insert(0, incom_region.columns[0])
incom_region

region     2013                    ... 2020              
                                          1        2        3  ...    2   3   4 total
0            Российская Федерация   21800.0  24990.0  25529.0  ...  NaN NaN NaN   NaN
1    Центральный федеральный округ  28189.0  34117.0  33746.0  ...  NaN NaN NaN   NaN
2           Белгородская область    19980.0  23494.0  23113.0  ...  NaN NaN NaN   NaN
3                Брянская область   16799.0  18762.0  18538.0  ...  NaN NaN NaN   NaN
4             Владимирская область  15802.0  17319.0  18270.0  ...  NaN NaN NaN   NaN
..                             ...      ...      ...      ...  ...  ...  ..  ..   ...
96            Магаданская область   38845.0  42256.0  41321.0  ...  NaN NaN NaN   NaN
97          Сахалинская область     34900.0  37268.0  37331.0  ...  NaN NaN NaN   NaN
98          Еврейская  авт.область  17314.0  20735.0  19411.0  ...  NaN NaN NaN   NaN
99            Чукотский  авт.округ  50547.0  54655.0  54356.0  ...  NaN NaN NaN   NaN
100       * Предварительные данные      NaN      NaN      NaN  ...  NaN NaN NaN   NaN

[101 rows x 41 columns]

In [211]:
incom_region.region = incom_region.region.str.strip().str.replace('область', 'обл.')

In [213]:
def get_income_region(x):
  year, period = x.RequestDate_year, x.RequestDate_quarter
  assert np.isin(x.LivingRegionName, incom_region.region)
  return incom_region[(incom_region.region == x.LivingRegionName)][(year,period)].iat[0]

get_income_region(df.iloc[0])
res = df.apply(get_income_region, axis=1)

In [216]:
res.isna().sum()

0